In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
import torch
import os

import sys
sys.path.insert(0, '../trainingdata')
from datasaver import *
sys.path.insert(0, '../testproblems/tdarcy')
from manufacturedsolutions import *
sys.path.insert(0, '../ml')
from modelloader import loadmodelfromlabel
from NeuralOperator import NeuralOperator
from customlosses import *
from quadrature import *
from customlayers import discretize_functions
from basisfunctions import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from NeuralOperator import NeuralOperator

logdir = '../../../nnlogs'
device = 'cuda:2'

sublogdir = 'tdd'

projection = loadmodelfromlabel(model=NeuralOperator, label='modelNGO', logdir=logdir, sublogdir=sublogdir, device=device)
projection.hparams['modeltype'] = 'projection'
projection.hparams['dtype'] = torch.float64
projection.__init__(projection.hparams)

FEM = loadmodelfromlabel(model=NeuralOperator, label='modelNGO', logdir=logdir, sublogdir=sublogdir, device=device)
FEM.hparams['modeltype'] = 'FEM'
FEM.hparams['dtype'] = torch.float64
FEM.hparams['Neumannseries'] = False
FEM.hparams['Neumannseries_order'] = None
FEM.__init__(FEM.hparams)

NN = loadmodelfromlabel(model=NeuralOperator, label='NN', logdir=logdir, sublogdir=sublogdir, device=device)

FNO = loadmodelfromlabel(model=NeuralOperator, label='FNO', logdir=logdir, sublogdir=sublogdir, device=device)

DeepONet = loadmodelfromlabel(model=NeuralOperator, label='DeepONet', logdir=logdir, sublogdir=sublogdir, device=device)

VarMiON = loadmodelfromlabel(model=NeuralOperator, label='VarMiON', logdir=logdir, sublogdir=sublogdir, device=device)

dataNGO = loadmodelfromlabel(model=NeuralOperator, label='dataNGO', logdir=logdir, sublogdir=sublogdir, device=device)

dataNGO_C = loadmodelfromlabel(model=NeuralOperator, label='dataNGO_C', logdir=logdir, sublogdir=sublogdir, device=device)

datafreeNGO = loadmodelfromlabel(model=NeuralOperator, label='datafreeNGO', logdir=logdir, sublogdir=sublogdir, device=device)

datafreeNGO_C = loadmodelfromlabel(model=NeuralOperator, label='datafreeNGO_C', logdir=logdir, sublogdir=sublogdir, device=device)

modelNGO = loadmodelfromlabel(model=NeuralOperator, label='modelNGO', logdir=logdir, sublogdir=sublogdir, device=device)

modelNGO_C = loadmodelfromlabel(model=NeuralOperator, label='modelNGO_C', logdir=logdir, sublogdir=sublogdir, device=device)

modeldict = {'Projection': projection,
             'FEM': FEM,
              'NN': NN,
              'FNO': FNO,
              'DeepONet': DeepONet,
             'VarMiON': VarMiON,
            #'Data NGO': dataNGO,
            'Data NGO': dataNGO_C,
             # 'Data-free NGO': datafreeNGO,
             'Data-free NGO': datafreeNGO_C,
             # 'Model NGO': modelNGO,
             'Model NGO': modelNGO_C}

/home/prins/st8/prins/phd/git/ngo-pde-gk/ml/quadrature.py:67: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  xis = np.vstack(map(np.ravel, grid)).T


In [7]:
N_samples = 10
n_timesteps = 100
t_values = np.linspace(0,n_timesteps*projection.hparams['Dt'],n_timesteps)

alpha = 0.05
q_low = alpha/2
q_high = 1 - q_low

L2_scaled_avg = np.zeros((len(modeldict),n_timesteps))
yerr = np.zeros((len(modeldict),2,n_timesteps))

size_title = 20
size_ticklabels = 15
size_axeslabels = 20

# quadrature = GaussLegendreQuadrature(n_elements=[1,3,3], Q=[3,30,30])
quadrature = UniformQuadrature(Q=[3,30,30])
x = quadrature.xi
w = quadrature.w
w = w*projection.hparams['Dt']

linestyles=['-','-','--','--','--','--','--','--','--','--','--','--','--','--','--']
colors = ['black','grey','C0','C1','C2','C3','C4','C5','C6','C7','C8']

linestyles=['-','-','--','--','--','--','--','--','--','--','--']
colors = ['black','grey','C3','C6','C4','C5','C0','C1','C2']

dataset = ManufacturedSolutionsSet(N_samples=N_samples, variables=['t','x','x'], l_min=[1,1,1], l_max=[1,1,1])
theta = dataset.theta
f = dataset.f
etat = dataset.etat
etab = dataset.etab
gl = dataset.gl
gr = dataset.gr
u = dataset.u
u0 = dataset.u0

m=0
for model in modeldict:
    print(str(model))
    modeldict[str(model)].timestep = 0
    x_scaled = np.copy(x)
    x_scaled[:,0] = x_scaled[:,0]*modeldict[str(model)].hparams['Dt']
    for i in range(n_timesteps):
        u_exact = modeldict[str(model)].compute_true_mass(u, x_scaled[:,0])
        u_hat = modeldict[str(model)].compute_mass(theta, f, etab, etat, gl, gr, u0, x[:,0], u)
        L2_scaled_array = weightedrelativeL2_set(w, u_hat, u_exact)
        L2_scaled_avg[m,i] = np.average(L2_scaled_array)
        q_l = np.quantile(L2_scaled_array, q_low)
        q_h = np.quantile(L2_scaled_array, q_high)
        yerr[m,:,i] = np.array([q_l, q_h])
        modeldict[str(model)].translate_quadrature_t()
        x_scaled[:,0] = x_scaled[:,0] + modeldict[str(model)].hparams['Dt']
        modeldict[str(model)].timestep += 1
    m+=1

fig, ax = plt.subplots(1,1, figsize=(6, 4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.4, hspace=0.4)

plots = []
m=0
ax.tick_params(axis='both', labelsize=size_ticklabels)
ax.set_xlabel(r'Time $t/T$', fontsize=size_axeslabels)
ax.set_ylabel(r'Rel. L2 error $\frac{||\hat{u}-u||_2}{||u||_2}$', fontsize=size_axeslabels)
ax.set_yscale('log')
# ax.set_xlim(0, 1.05)
ax.set_ylim(1e-6,1e2)
ax.set_yticks([1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2])
ax.grid()
# ax.set_title('FNO', fontsize=size_title)
for model in modeldict:
    # plots.append(ax.errorbar(t_values, L2_scaled_avg[m], yerr=yerr[m], fmt=".-", capsize=6, ms=15, label=str(model), color=colors[m], linestyle=linestyles[m]))
    plots.append(ax.plot(t_values, L2_scaled_avg[m], label=str(model), color=colors[m], linestyle=linestyles[m])[0])
    ax.fill_between(t_values, yerr[m,0,:], yerr[m,1,:], alpha=0.1, color=colors[m])
    m+=1
# plt.legend(fontsize=10, ncols=1)
legend1 = ax.legend(handles=plots[0:2],fontsize=10, ncols=1, loc='lower right')
plt.gca().add_artist(legend1)
legend2 = ax.legend(handles=plots[2:],fontsize=10, ncols=2, loc='upper right')
plt.gca().add_artist(legend2)

plt.savefig("errorovertime.svg", bbox_inches='tight')#, transparent=True)

plt.show()

Projection
(2700,)
(900, 3)


ValueError: could not broadcast input array from shape (2700,) into shape (900,)